Code for the creation and testing of our LDA model for complexity

All of the cleaned and processed data has been stored in Drive.

All you need are the Shuffled.csv, final_data.csv, and doc_term_matrix.pkl files for running the last section

In [ ]:
# Use if you want to run with GPU
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# Connect to Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# install first or the cleantext library won't load
!pip install cleantext

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.stats import entropy
# Uncomment when you need to re-clean the data
# import cleantext
import re
from time import perf_counter

From here ONLY RUN THE LAST CELL OF CODE. If you run other sections/lines you risk messing up the data files and will have to run everything over again. 

WARNING:

DO NOT RUN the Document Matrix Creation code in colab. It exceeds 12gb of ram and will crash the colab process. I had to run it on my laptop locally

The next 2 sections deal with data cleaning

In [ ]:
dataset = pd.read_csv('/content/gdrive/MyDrive/CS_490_DATA/final_data.csv').dropna()


# This section does everything but lemmatize the text
t1_start = perf_counter()
with tf.device('/device:GPU:0'): 
    for opinion in range(len(dataset)):

        text = dataset["Opinion"].iloc[opinion]
        text = re.sub(r'[^\w\s]', '', text)

        clean = cleantext.clean(text, clean_all= False, # Execute all cleaning operations
          extra_spaces=True ,  # Remove extra white spaces
          stemming= False, # Stem the words
          stopwords=True ,# Remove stop words
          lowercase=True ,# Convert to lowercase
          numbers=True ,# Remove all digits
          punct=True)
        
        dataset.at[opinion, 'Opinion'] = clean
        
    print("cleaning done")

t1_end = perf_counter()

print("Elapsed time:", t1_end - t1_start)

dataset.to_csv("/content/gdrive/MyDrive/CS_490_DATA/Cleaned_data.csv", index=False)


In [ ]:
# This section lemmatizes the text using spacy library

dataset_clean = pd.read_csv('/content/gdrive/MyDrive/CS_490_DATA/Cleaned_data.csv').dropna()

import spacy
nlp = spacy.load('en_core_web_sm')


cleaned_data = dataset_clean
with tf.device('/device:GPU:0'): 
  for i in range(len(cleaned_data)):
      text = str(cleaned_data["Opinion"].iloc[i])
      # Create a Doc object
      doc = nlp(text, disable=["tagger", "parser", "ner","attribute_ruler"]) # disable removes unecessary functions that increase function time
      # Create list of tokens from given Doc object
      sentence = " ".join([token.lemma_ if token.lemma_ != '-PRON-' else token.lower_ for token in doc])
      cleaned_data.at[i,'Opinion'] = sentence
  print("lemma done")
  cleaned_data.to_csv('/content/gdrive/MyDrive/CS_490_DATA/Cleaned_2_data.csv')

In [ ]:
dataset_to_be_shuffled = pd.read_csv('/content/gdrive/MyDrive/CS_490_DATA/Cleaned_2_data.csv').dropna()

shuffled = dataset_to_be_shuffled.sample(frac=1)
shuffled = shuffled.drop(columns = ["Unnamed: 0","Unnamed: 0.1"])
shuffled.to_csv("Shuffled.csv")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.9, min_df=.05, stop_words='english')
doc_term_matrix = count_vect.fit_transform(shuffled['Opinion'].values.astype('U'))
print(doc_term_matrix.shape)
print("doc_term_done")

import joblib

# Save Document Term Matrix as a picle file
joblib.dump(doc_term_matrix, "doc_term_matrix.pkl")

In [ ]:
final_dataset = pd.read_csv('/content/gdrive/MyDrive/CS_490_DATA/final_data.csv').dropna()


# Load in the Document Term Matrix
import joblib
doc_term_matrix = joblib.load("/content/gdrive/MyDrive/CS_490_DATA/doc_term_matrix.pkl")


# Split into traing and testing set
training = doc_term_matrix[0:4800]
test = doc_term_matrix[4800:]

with tf.device('/device:GPU:0'):  
  from sklearn.decomposition import LatentDirichletAllocation

  t3_start = perf_counter()

  LDA = LatentDirichletAllocation(n_components=20)
  LDA.fit(training)
  print(LDA.perplexity(test))
  topic_values = LDA.transform(doc_term_matrix)
  calc = []
  for i in range(len(topic_values)):
      entr = entropy(topic_values[i],base=20)
      calc.append(entr*100)


final_dataset['Topic_Score'] = calc
print(final_dataset[['Case_id','Topic_Score']])

t3_end = perf_counter()
print("LDA Elapsed time:", t3_end - t3_start)

From here on down is code for experiemntal results. Two topic visualization, one for the initial and best model. Also, some basic statistical results like the mean and max entropy scores for each court.

In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=f34dc8aa63475fc6d4693e1e0a149f7c650ad509c0c02fe6e65bf19179aca2c3
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [ ]:
import joblib
count_vect = joblib.load('/content/gdrive/MyDrive/CS_490_DATA/count_vect.pkl')
doc_term_matrix = joblib.load("/content/gdrive/MyDrive/CS_490_DATA/doc_term_matrix.pkl")

lda_init = joblib.load("/content/gdrive/MyDrive/CS_490_DATA/lda_init.pkl")
lda_best = joblib.load("/content/gdrive/MyDrive/CS_490_DATA/lda_best.pkl")

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

pyLDAvis.enable_notebook()

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [ ]:
vis1 = pyLDAvis.sklearn.prepare(lda_init, doc_term_matrix, count_vect)
vis1

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.126483 -0.041626       1        1  13.933003
3      0.006573  0.024471       2        1  12.591102
7      0.061090  0.105456       3        1  10.656271
0      0.061192  0.051088       4        1  10.331678
8     -0.053373 -0.039252       5        1  10.059817
4      0.078359 -0.136470       6        1   9.583431
6     -0.125784 -0.017658       7        1   9.402366
5      0.017101  0.127945       8        1   8.761089
9     -0.051220  0.005456       9        1   7.368871
1      0.132545 -0.079410      10        1   7.312371, topic_info=           Term          Freq         Total Category  logprob  loglift
1817   sentence  35799.000000  35799.000000  Default  30.0000  30.0000
1491  plaintiff  38071.000000  38071.000000  Default  29.0000  29.0000
324       claim  65447.000000  65447.000000  Default  28.0000  28.0000
1383    officer  20559.000000  20559.000000  Default  27.0000  27.0000
1305     motion  34063.000000  34063.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
1999         th   3774.686524  47640.239122  Topic10  -5.0392   0.0802
1305     motion   3347.965665  34063.493456  Topic10  -5.1591   0.2957
1155        law   3362.344294  42708.261899  Topic10  -5.1548   0.0738
1397    opinion   2567.780499  18561.097601  Topic10  -5.4244   0.6376
1101      issue   2466.091067  32510.284419  Topic10  -5.4648   0.0367

[625 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
2         1  0.025974    14
2         4  0.344968    14
2         5  0.599026    14
2         6  0.004058    14
2         7  0.000812    14
...     ...       ...   ...
2184      6  0.002770  york
2184      7  0.076048  york
2184      8  0.056532  york
2184      9  0.010576  york
2184     10  0.076803  york

[4020 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 8, 1, 9, 5, 7, 6, 10, 2])

In [ ]:
vis2 = pyLDAvis.sklearn.prepare(lda_best, doc_term_matrix, count_vect)
vis2

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
6     -0.099026  0.051523       1        1  7.649610
18     0.121527  0.016185       2        1  7.542332
3     -0.034072 -0.187018       3        1  6.778010
17    -0.078509 -0.016457       4        1  6.289538
10    -0.114349  0.000420       5        1  6.138348
16    -0.085299  0.036752       6        1  6.027208
19    -0.007945  0.077686       7        1  5.810140
12     0.116877 -0.021220       8        1  5.448921
11    -0.033921  0.033650       9        1  5.322849
13     0.127134  0.029436      10        1  4.898049
4     -0.099496  0.101113      11        1  4.639731
7      0.129942  0.074020      12        1  4.445363
1      0.082809  0.035145      13        1  4.406571
8      0.035162  0.013668      14        1  4.307417
5      0.028695 -0.029125      15        1  4.255212
2      0.037789 -0.032866      16        1  3.704306
0     -0.030521 -0.204067      17        1  3.648649
9     -0.098639  0.041531      18        1  3.211274
15     0.071922 -0.037188      19        1  2.765171
14    -0.070079  0.016810      20        1  2.711300, topic_info=            Term          Freq         Total Category  logprob  loglift
1817    sentence  35665.000000  35665.000000  Default  30.0000  30.0000
1491   plaintiff  37354.000000  37354.000000  Default  29.0000  29.0000
901   government  37208.000000  37208.000000  Default  28.0000  28.0000
1383     officer  20385.000000  20385.000000  Default  27.0000  27.0000
2042       trial  23349.000000  23349.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
1397     opinion   1324.543570  18529.889755  Topic20  -5.0943   0.9694
324        claim   1717.718979  64523.518116  Topic20  -4.8343  -0.0183
1782        rule   1429.484817  41952.792061  Topic20  -5.0180   0.2285
1442       party   1168.094724  24675.125063  Topic20  -5.2200   0.5573
645     document   1185.338361  70198.768397  Topic20  -5.2053  -0.4736

[1151 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0         1  0.106713    10
0         2  0.001309    10
0         3  0.549276    10
0         4  0.001309    10
0         5  0.014403    10
...     ...       ...   ...
2184     15  0.000623  york
2184     16  0.008093  york
2184     17  0.003984  york
2184     18  0.019672  york
2184     20  0.010459  york

[11966 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 19, 4, 18, 11, 17, 20, 13, 12, 14, 5, 8, 2, 9, 6, 3, 1, 10, 16, 15])

In [ ]:
dataset = pd.read_csv('/content/gdrive/MyDrive/CS_490_DATA/Shuffled.csv').dropna()

with tf.device('/device:GPU:0'):  
  from sklearn.decomposition import LatentDirichletAllocation

  topic_values = lda_best.transform(doc_term_matrix)
  calc = []
  for i in range(len(topic_values)):
      entr = entropy(topic_values[i],base=20) # make the base = n_components to normalize results between 0-1
      calc.append(entr*100)


dataset['Complexity'] = calc
print(dataset[['Case_id','Complexity']])

       Case_id  Complexity
0     20-01615   59.400737
1     20-56075   38.838116
2     20-02943   34.890994
3     20-50157   57.756222
4     19-03586   45.178934
...        ...         ...
5995  19-01639   67.555699
5996  20-10479   44.450517
5997  20-05027   67.370377
5998  20-02615   45.054450
5999  16-04831   48.812198

[6000 rows x 2 columns]


In [ ]:
print(dataset["Complexity"].describe())

count    6000.000000
mean       39.383367
std        17.009937
min         0.111783
25%        28.200564
50%        41.806555
75%        52.204849
max        78.863183
Name: Complexity, dtype: float64


In [ ]:
print(dataset.groupby("Court_id")["Complexity"].mean().sort_values(ascending=False))

Court_id
2     40.729527
1     40.723258
4     40.058614
12    39.573068
10    39.551457
9     39.496798
3     39.296477
7     39.269191
8     39.125632
5     38.949504
11    38.396673
6     37.430201
Name: Complexity, dtype: float64


In [ ]:
print(dataset.groupby("Complexity")["Complexity"].max().sort_values(ascending=False))

Complexity
78.863183    78.863183
77.970967    77.970967
77.119405    77.119405
76.816179    76.816179
76.776103    76.776103
               ...    
0.299773      0.299773
0.233549      0.233549
0.228016      0.228016
0.115672      0.115672
0.111783      0.111783
Name: Complexity, Length: 5572, dtype: float64


In [ ]:
print(dataset[dataset["Complexity"] == dataset["Complexity"].max()])

      Former_Index  Court_id   Case_id  \
2844           784         2  20-01228   

                                                Opinion  Complexity  
2844  case document page l doe v trump corporation u...   78.863183  


In [ ]:
print(dataset[dataset["Complexity"] == dataset["Complexity"].min()])

      Former_Index  Court_id   Case_id  \
231           2978         6  20-04010   
1147          1244         3  19-03150   
1306           662         2  17-02898   
1747          1785         4  19-07508   
2185          5492        11  19-13315   
2444          5653        12  18-03016   
3353           212         1  19-02095   
3523          1145         3  19-03042   
3574          3958         8  12-02716   
4263          3398         7  20-02919   
4921          5004        11  18-12104   

                                                Opinion  Complexity  
231   case document file page recommend publication ...    0.111783  
1147  case document page date file precedential unit...    0.111783  
1306  case document page martinez de artiga v barr k...    0.111783  
1747  usca appeal doc file pg unpublished unite stat...    0.111783  
2185  case date file page publish unite state court ...    0.111783  
2444  usca case document file page unite state court...    0.111783  
3353 

In [ ]:
import plotly.express as px

/usr/local/lib/python3.7/dist-packages/yaml/constructor.py:126: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(key, collections.Hashable):
/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.divide(0.4, 1, casting="unsafe", dtype=np.float),
/usr/local/lib/python3.7/dist-packages/scipy/io/matlab/mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any beha

In [ ]:
fig = px.box(dataset, y="Complexity",title='Complexity Box-Plot')
fig.show()

In [ ]:
fig = px.box(dataset, x="Court_id", y="Complexity",color="Court_id",title='Complexity By Court (Box-Plot)')
fig.show()

In [ ]:
fig = px.histogram(dataset, x="Complexity",title="Complexity Histogram")
fig.show()

In [ ]:
import plotly.figure_factory as ff

In [ ]:
x = dataset["Complexity"]
hist_data = [x]
group_labels = ['distplot'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels)
fig.show()

In [ ]:
x = []
y = []
for i in range(1,13):
  x.append(dataset[dataset["Court_id"] == i]["Complexity"].mean())
  y.append(i)


In [ ]:
dat = pd.DataFrame({'Complexity': x,
                    'Court_id': y})

fig = px.bar(dat, x='Court_id', y='Complexity',color="Court_id",title="Average Complexity by Court")

fig.show()

In [ ]:
data2 = pd.read_csv('/content/gdrive/MyDrive/CS_490_DATA/cleaned_2_with_pol_and_sub.csv').dropna()
print(data2)

In [ ]:
dataset = dataset.sort_values(by=["Former_Index"],ascending= True).reset_index(drop=True)

In [ ]:
print(dataset)

In [ ]:
dataset["Polarity"] = data2["polarity"]
dataset["Subjectivity"] = data2["subjectivity"]

dataset["Uncleaned_Opinion"] = pd.read_csv("/content/gdrive/MyDrive/CS_490_DATA/final_data.csv")["Opinion"]

In [ ]:
print(dataset)

In [ ]:

fig = px.scatter_matrix(dataset,
    dimensions=["Complexity", "Polarity", "Subjectivity"],title="Correlation Plot of All Variables of Interest")
fig.show()

In [ ]:
dataset = dataset.drop(["Former_Index"],axis=1)
for i in range(len(dataset)):
  words = dataset["Case_id"].iloc[i].split('-')
  string = words[0] + '-' + words[1][1:]
  dataset["Case_id"][i] = string


In [ ]:
print(dataset)

      Court_id  Case_id                                            Opinion  \
0            1  21-1566  case document page date file entry i would uni...   
1            1  19-1622  case document page date file entry i would uni...   
2            1  20-1434  case document page date file entry i would uni...   
3            1  19-2104  case document page date file entry i would uni...   
4            1  20-1942  case document page date file entry i would uni...   
...        ...      ...                                                ...   
5995        12  21-5080  usca case document file page unite state court...   
5996        12  20-1045  usca case document file page unite state court...   
5997        12  19-5199  usca case document file page unite state court...   
5998        12  20-7110  usca case document file page unite state court...   
5999        12  18-3070  usca case document file page unite state court...   

      Complexity  Polarity  Subjectivity  \
0      25.709101 -0

In [ ]:
dataset.to_csv("/content/gdrive/MyDrive/CS_490_DATA/Court_Complexity_Polarity_Subjectivity_data.csv",index=False)